In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib
import re
import string

In [6]:
fake=pd.read_csv('Fake.csv')
true=pd.read_csv('True.csv')

In [11]:
true.head()

title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  class  
0  December 31, 2017       1  
1  December 29, 2017       1  
2  December 31, 2017       1  
3  December 30, 2017       1  
4  December 29, 2017       1

In [10]:
fake.head()


title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  class  
0  December 31, 2017      0  
1  December 31, 2017      0  
2  December 30, 2017      0  
3  December 29, 2017      0  
4  December 25, 2017      0

In [9]:
fake['class'] = 0
true['class'] = 1

In [12]:
data=pd.concat([fake, true], axis=0)

In [19]:
data.sample(10)

text  class
28568  (Reuters) - Republicans in the U.S. House of R...      1
17860  Let s hope this angry leftist mayor doesn t pr...      0
20484  Yet another assault on a Trump supporter by a ...      0
27518  WASHINGTON (Reuters) - A U.S. appeals court on...      1
12751  Democrats are on the move. They are panicking ...      0
29948  WASHINGTON (Reuters) - Russian President Vladi...      1
35317  CUCUTA, Colombia (Reuters) - Thousands of impo...      1
18005  Two days ago, Cher, a leftist agitator, and Tr...      0
18806  Yesterday, Antifa (anarchists and free-speech ...      0
17524  It s beyond sad to watch these young men sitti...      0

In [14]:
data=data.drop(['title','subject','date'], axis=1)

In [16]:
data.reset_index( inplace=True)

In [18]:
data.drop(['index'], axis=1, inplace=True)

In [20]:
data.sample(5)

text  class
5550   Newly released video shows a group of Texas in...      0
39187  MOSCOW (Reuters) - Russian Tu-22M3 long-range ...      1
38252  PHNOM PENH (Reuters) - Cambodia s main opposit...      1
28894  NOGALES, Arizona (Reuters) - For up to 16 hour...      1
5362   Infamous Italian dictator Benito Mussolini ant...      0

In [30]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\[.*?\]', '', text)  # Remove content in square brackets
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\n', ' ', text)  # Replace newlines with space
    text = re.sub(r'\W+', ' ', text)  # Replace non-word characters with space
    text = text.strip()  # Remove leading/trailing whitespace
    return text

In [ ]:
data["text"] = data["text"].apply(clean_text)

In [33]:
x=data["text"]
y=data["class"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [34]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)


In [35]:
lr=LogisticRegression()
lr.fit(xv_train, y_train)

LogisticRegression()

In [36]:
predictions = lr.predict(xv_test)
lr.score(xv_test, y_test)


0.9864142538975501

In [37]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4733
           1       0.98      0.99      0.99      4247

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [38]:
joblib.dump(lr, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']